# 데이터 전처리 (1) - 행 열 재배치

- 행/열(변수)에 대한 재배치, 이름 변경
- 정렬
- 특정 데이터 필터링 - 조회
- 행 혹은 열 추가, 삭제
- 파생변수 추가

### 1. 분석 준비

#### 1) 패키지 참조

In [56]:
from pandas import read_excel
from pandas import Series, CategoricalDtype
import numpy as np

#### 2) 데이터 가져오기

In [57]:
df = read_excel("https://data.hossam.kr/C02/pre_sample.xlsx", index_col="이름")
df

,학년,성별,국어,영어,수학,과학
이름,,,,,,
철수,1,남자,98,77,88,64
영희,2,여자,88,120,62,72
민철,1,남자,92,70,83,79
수현,3,여자,63,60,31,71
호영,4,남자,75,50,90,88
영호,4,남자,80,88,91,72
용식,2,남자,82,88,79,90
나영,1,여자,90,92,81,95
석영,1,남자,91,90,89,80


#### 3) 데이터 확인하기

In [58]:
# 데이터의 크기
df.shape

# 각 변수의 데이터 타입 확인
df.dtypes

# 데이터 타입 변경
# : 일괄 변경 후 새로운 데이터 프레임 리턴받기
# : 결과가 적용된 복사본 리턴
df2 = df.astype({'국어': 'float', '영어': 'float', '수학': 'str', '과학': 'bool'})
df2
df2.dtypes

# 원본 확인 --> .astype() 함수는 결과가 적용된 복사본을 리턴한다 (원본은 변하지 않음)
df.dtypes

# 개별 컬럼단위로 수정하여 원본에 적용하기: int, float, str, bool
df['수학'] = df["수학"].astype('int')
df['성별'] = df["성별"].astype('str')
df.dtypes

# 범주형 데이터 만들기

# 1) 데이터프레임 전체에 적용
df3 = df.astype({'학년': 'category', '성별': 'category'})
df3.dtypes

# 2) 컬럼 단위로 적용
df['학년'] = df['학년'].astype('category')
df['성별'] = df['성별'].astype('category')
df.dtypes

# 3) 서열척도로 변경 (순서 있는 범주형)
df4 = df.astype({"학년": CategoricalDtype(categories=[1, 2, 3, 4], ordered=True)})
df4.dtypes

df['학년'] = df['학년'].astype(
        CategoricalDtype(categories=[1, 2, 3, 4], ordered=True))
df.dtypes

학년    category
성별    category
국어       int64
영어       int64
수학       int64
과학       int64
dtype: object

### 2. 행/열(변수)에 대한 재배치, 이름 변경

### 1) 열 순서 변경
- `데이터프레임.reindex(columns=['인덱스1', '인덱스2', '인덱스3', '인덱스4', ...])`
- 지정된 순서대로 열이 재정렬 된 결과 반환
- 원본은 변화 없음
- 결과가 적용된 복사본 생성
- 재배치 되지 못한 열은 제외됨

In [59]:
df1 = df.reindex(columns = ['국어', '수학', '과학', '영어'])
df1

,국어,수학,과학,영어
이름,,,,
철수,98,88,64,77
영희,88,62,72,120
민철,92,83,79,70
수현,63,31,71,60
호영,75,90,88,50
영호,80,91,72,88
용식,82,79,90,88
나영,90,81,95,92
석영,91,89,80,90


#### 2) 행 순서 변경 
- `데이터프레임.reindex(index=['인덱스1', '인덱스2', '인덱스3', '인덱스4', ...])`
- 지정된 순서대로 행이 재정렬 된 결과 반환
- 원본은 변화 없음
- 결과가 적용된 복사본 생성
- 재배치 되지 못한 행은 제외됨

In [60]:
df2 = df.reindex(index=['석영', '호영', '민철', '수현'])
df2

,학년,성별,국어,영어,수학,과학
이름,,,,,,
석영,1,남자,91,90,89,80
호영,4,남자,75,50,90,88
민철,1,남자,92,70,83,79
수현,3,여자,63,60,31,71


#### 3) 행, 열의 이름 변경
- 기존이름: 새이름 형식의 딕셔너리로 지정
- columns와 index중 변경을 원하는 하나만 설정 가능
- 원본은 변화 없음
- 결과가 적용된 복사본 생성
- 이름이 변경되지 않은 항목이 결과에서 제외되지는 않음
- 원본에 즉시 반영해야 할 경우 `inplace=True` 파라미터 적용

In [61]:
df3 = df.rename(
    columns={'국어': 'kor', '영어': 'eng', '수학': 'math', '과학': 'sci'},
    index={'석영': 'suk', '호영': 'ho', '민철': 'min', '수현': 'su'}
)
df3

,학년,성별,kor,eng,math,sci
이름,,,,,,
철수,1,남자,98,77,88,64
영희,2,여자,88,120,62,72
min,1,남자,92,70,83,79
su,3,여자,63,60,31,71
ho,4,남자,75,50,90,88
영호,4,남자,80,88,91,72
용식,2,남자,82,88,79,90
나영,1,여자,90,92,81,95
suk,1,남자,91,90,89,80


### 3. 정렬
- `.sort()`
- 특정 변수(열, 컬럼)를 기준으로 하여 행 단위로 정렬

#### 1) 변수에 대한 오름차순 정렬 (ASC)
- `데이터프레임.sort_values(['컬럼명'])`

In [62]:
# 단일 컬럼에 대한 정렬: 원본에 즉시 반영해야할 경우 `inplace = True` 파라미터를 적용
df4 = df.sort_values('국어')
df4

# 복수 컬럼에 대한 정렬: 정렬 기준이 되는 컬럼 이름을 리스트로 설정하면 첫 번째 컬럼의 값이 동일한 경우 두 번째 컬럼을 기준으로 정렬한다.
df5 = df.sort_values(['학년', '영어'])
df5

,학년,성별,국어,영어,수학,과학
이름,,,,,,
민철,1,남자,92,70,83,79
철수,1,남자,98,77,88,64
석영,1,남자,91,90,89,80
나영,1,여자,90,92,81,95
용식,2,남자,82,88,79,90
영희,2,여자,88,120,62,72
수현,3,여자,63,60,31,71
호영,4,남자,75,50,90,88
영호,4,남자,80,88,91,72


#### 2) 변수에 대한 내림차순 정렬(DESC)
- `데이터프레임.sort_values(['컬럼명'], ascending = False)`
- `ascending = False` 파라미터를 추가한다 (기본값 = True, 오름차순)

In [63]:
df6 = df.sort_values('국어', ascending=False)
df6

,학년,성별,국어,영어,수학,과학
이름,,,,,,
철수,1,남자,98,77,88,64
민철,1,남자,92,70,83,79
석영,1,남자,91,90,89,80
나영,1,여자,90,92,81,95
영희,2,여자,88,120,62,72
용식,2,남자,82,88,79,90
영호,4,남자,80,88,91,72
호영,4,남자,75,50,90,88
수현,3,여자,63,60,31,71


#### 3) 인덱스를 기준으로 하는 정렬
- .sort_index()
- 인덱스는 하나만 존재하므로 열 이름을 지정할 필요가 없음
- 정렬 방향을 위한 `ascending = False` 설정 시 내림차순으로 정렬됨
- `inplace=True`를 적용할 경우 원본에 즉시 반영

In [64]:
df7 = df.sort_index()
df7

,학년,성별,국어,영어,수학,과학
이름,,,,,,
나영,1,여자,90,92,81,95
민철,1,남자,92,70,83,79
석영,1,남자,91,90,89,80
수현,3,여자,63,60,31,71
영호,4,남자,80,88,91,72
영희,2,여자,88,120,62,72
용식,2,남자,82,88,79,90
철수,1,남자,98,77,88,64
호영,4,남자,75,50,90,88


### 4. 특정 데이터 필터링
- .query('조건')
- .filter(['컬럼'])

#### 1) 행 단위 필터링 (조건 검색)
- .query('행 조건')

In [65]:
# 특정 조건에 맞는 행 조회
r1 = df.query('국어 > 80')
r1

# AND 조건
r2 = df.query('국어 > 80 and 수학 > 80')
r2

# # OR 조건
# r3 = df.query('국어 > 80 or 수학 > 80')
# r3

# 조건값을 변수로 적용하기
# point = int(input('점수를 입력하세요: '))
# print("입력한 점수인 %d점 보다 높은 학생은 다음과 같습니다." % point)
# r4 = df.query('국어 > @point')
# r4

,학년,성별,국어,영어,수학,과학
이름,,,,,,
철수,1,남자,98,77,88,64
민철,1,남자,92,70,83,79
나영,1,여자,90,92,81,95
석영,1,남자,91,90,89,80


#### 2) 열 단위 필터링
- .filter(['컬럼명'])

In [66]:
fl = df.filter(['국어', '영어'])
fl

,국어,영어
이름,,
철수,98,77
영희,88,120
민철,92,70
수현,63,60
호영,75,50
영호,80,88
용식,82,88
나영,90,92
석영,91,90


### 5. 행/열 추가, 삭제
- .loc[''] = 리스트/딕셔너리/시리즈

#### 1) 행 추가
- 원본 자체 수정

In [71]:
# 리스트로 추가
# : 추가될 행의 인덱스 이름을 지정해야 함
# : 리스트로 추가할 경우 dataframe의 컬럼 순서에 맞게 지정해야 함
# : 누락되는 값이 있거나 값의 수가 초과할 경우 에러
dff = df.copy()
dff.loc['정호'] = [2, '남자', 90, 80, 70, 60]
dff

# 딕셔너리로 추가
# : 컬럼 순서는 상관 없다
# : 누락되는 값이 있거나 값의 수가 초과할 경우 에러
dff.loc['민정']= {'성별': '여자', '학년': 4, '국어': 81, '영어': 72, '수학': 84, '과학': 90}
dff

# 시리즈 객체로 추가
# : 시리즈 객체의 index가 지정되지 않은 경우 값 추가 불가능
s = Series([2, '남자', 90, 80, 70, 60], index=['학년', '성별', '국어', '영어', '수학', '과학'])
dff.loc['형석'] = s
dff

# 빈값이 있는 경우
# : Series 객체 형태만 가능
s = Series([2, '남자', 93, 71, 68], index=['학년', '성별', '국어', '수학', '과학'])
dff.loc['석호'] = s
dff

# 기존의 행 복사하여 추가
dff.loc['철민'] = df.loc['철수']
dff

# 멀티 인덱스 데이터프레임에 추가
multiindex = dff.set_index(['학년'], append=True)
multiindex

,,성별,국어,영어,수학,과학
이름,학년,,,,,
철수,1,남자,98,77.0,88,64
영희,2,여자,88,120.0,62,72
민철,1,남자,92,70.0,83,79
수현,3,여자,63,60.0,31,71
호영,4,남자,75,50.0,90,88
영호,4,남자,80,88.0,91,72
용식,2,남자,82,88.0,79,90
나영,1,여자,90,92.0,81,95
석영,1,남자,91,90.0,89,80


#### 2) 행 삭제
- 추가는 원본 자체가 변하지만 삭제는 결과가 반영된 새로운 복사본이 생성됨
- `.drop()` 메서드로 `inplace=True` 파라미터 적용 가능
- data = data_pre.drop(data_pre.query("필드 > 40000").index)

In [68]:
# 단일 행 삭제
k1 = df.drop('철민')
k1

# 다중 행 삭제
k2 = df.drop(['석영', '정호', '민정', '형석'])
k2

# index에 대한 indexing으로 삭제
k3 = df.drop(df.index[0])
k3

# index에 대한 slicing으로 삭제
k4 = df.drop(df.index[0:10])
k4

# 특정 값 연산으로 삭제
k4_1 = df.drop(df.query("수학 > 80").index)
k4_1

KeyError: "['철민'] not found in axis"

#### 3) 열 추가
> k4에서 이어서 진행

In [ ]:
# 리스트로 추가
k4['한국사'] = [72, 56, 90, 88]
k4

# 딕셔너리로 추가
k4['세계사'] = {"민정": 90, "형석": 88, "석호": 72, "철민": 56}
k4

# Series 객체로 추가
s = Series([72, 56, 90, 88], index=['석호', '철민', '민정', '형석'])
k4['윤리'] = s
k4

# 단일 값 추가
k4['체육'] = 100
k4

,학년,성별,국어,영어,수학,과학,한국사,세계사,윤리,체육
이름,,,,,,,,,,
민정,4,여자,81,72.0,84,90,72,90,90,100
형석,2,남자,90,80.0,70,60,56,88,88,100
석호,2,남자,93,NaN,71,68,90,72,72,100
철민,1,남자,98,77.0,88,64,88,56,56,100


- 특정 컬럼 인덱스에 추가
    - `df.insert(n, 'name_of_column', value)`
        - n = 넣고자 하는 컬럼 인덱스 위치 (정수)
        - value = 넣고자 하는 컬럼 값들
            - 여기에 ''을 넣으면 빈 값이 됨

In [ ]:
df.insert(0,'name_of_column','')
df['name_of_column'] = value

# 아니면

df.insert(0,'name_of_column',value)

NameError: name 'value' is not defined

#### 4) 열 삭제
- `.drop('컬럼명', axis=1)` 또는 `.drop('컬럼명', axis='columns')`
- 다수의 컬럼일 경우 리스트로 입력

In [ ]:
print(df)

# 단일 열 삭제
y1 = df.drop('성별', axis = 'columns')
y1

    학년  성별  국어     영어  수학  과학
이름                           
철수   1  남자  98   77.0  88  64
영희   2  여자  88  120.0  62  72
민철   1  남자  92   70.0  83  79
수현   3  여자  63   60.0  31  71
호영   4  남자  75   50.0  90  88
영호   4  남자  80   88.0  91  72
용식   2  남자  82   88.0  79  90
나영   1  여자  90   92.0  81  95
석영   1  남자  91   90.0  89  80
정호   2  남자  90   80.0  70  60
민정   4  여자  81   72.0  84  90
형석   2  남자  90   80.0  70  60
석호   2  남자  93    NaN  71  68
철민   1  남자  98   77.0  88  64


,학년,국어,영어,수학,과학
이름,,,,,
철수,1,98,77.0,88,64
영희,2,88,120.0,62,72
민철,1,92,70.0,83,79
수현,3,63,60.0,31,71
호영,4,75,50.0,90,88
영호,4,80,88.0,91,72
용식,2,82,88.0,79,90
나영,1,90,92.0,81,95
석영,1,91,90.0,89,80


### 6. 파생변수 추가
- 기존 변수들의 조합, 변형 혹은 계산을 통해 얻어지는 새로운 변수
#### 1) 데이터 새로 가져오기

In [ ]:
df = read_excel("https://data.hossam.kr/C02/pre_sample.xlsx", index_col='이름')
df

,학년,성별,국어,영어,수학,과학
이름,,,,,,
철수,1,남자,98,77,88,64
영희,2,여자,88,120,62,72
민철,1,남자,92,70,83,79
수현,3,여자,63,60,31,71
호영,4,남자,75,50,90,88
영호,4,남자,80,88,91,72
용식,2,남자,82,88,79,90
나영,1,여자,90,92,81,95
석영,1,남자,91,90,89,80


#### 2) 컬럼 간의 연산

In [ ]:
df1 = df.copy()
df1['직접합계'] = df1['국어'] + df1['영어'] + df1['수학'] + df1['과학']
df1


,학년,성별,국어,영어,수학,과학,직접합계
이름,,,,,,,
철수,1,남자,98,77,88,64,327
영희,2,여자,88,120,62,72,342
민철,1,남자,92,70,83,79,324
수현,3,여자,63,60,31,71,225
호영,4,남자,75,50,90,88,303
영호,4,남자,80,88,91,72,331
용식,2,남자,82,88,79,90,339
나영,1,여자,90,92,81,95,358
석영,1,남자,91,90,89,80,350


#### 3) 기술 통계값 관련 함수별 사용한 파생변수
- 합계, 평균, 최대, 최소, 중앙값, 분산, 표준편차 등
- `데이터프레임.iloc[]` 와 `데이터프레임.loc[]`
    - iloc : index-location (인덱스번호 기반)
    - loc : location (이름기반)

In [ ]:
# 합계
df2 = df.copy()
df2['합계'] = df2[df2.columns[2:6]].sum(axis=1)
df2

,학년,성별,국어,영어,수학,과학,합계
이름,,,,,,,
철수,1,남자,98,77,88,64,327
영희,2,여자,88,120,62,72,342
민철,1,남자,92,70,83,79,324
수현,3,여자,63,60,31,71,225
호영,4,남자,75,50,90,88,303
영호,4,남자,80,88,91,72,331
용식,2,남자,82,88,79,90,339
나영,1,여자,90,92,81,95,358
석영,1,남자,91,90,89,80,350


- `데이터프레임.iloc[]`로 데이터 추출 + 기술 통계값 관련 함수 사용

In [ ]:
print(df)

# 0번째 행 데이터만 가져오기
df.iloc[0]

# 1~4번째 전까지 행만 가져오기
df.iloc[1:4]

# 1번째 행의 2번째 열 데이터
df.iloc[1, 2]

# 모든 행에서 3번째 열만 가져오기
df.iloc[:, 3]

# 모든 행에 대해서 2번째-끝까지 열을 추출하여 합계
df.iloc[:, 2:].sum(axis=1)

    학년  성별  국어   영어  수학  과학
이름                         
철수   1  남자  98   77  88  64
영희   2  여자  88  120  62  72
민철   1  남자  92   70  83  79
수현   3  여자  63   60  31  71
호영   4  남자  75   50  90  88
영호   4  남자  80   88  91  72
용식   2  남자  82   88  79  90
나영   1  여자  90   92  81  95
석영   1  남자  91   90  89  80


이름
철수    327
영희    342
민철    324
수현    225
호영    303
영호    331
용식    339
나영    358
석영    350
dtype: int64

In [ ]:
df3 = df.copy()
df3['합계'] = df3.iloc[:, 2:].sum(axis=1)
df3

,학년,성별,국어,영어,수학,과학,합계
이름,,,,,,,
철수,1,남자,98,77,88,64,327
영희,2,여자,88,120,62,72,342
민철,1,남자,92,70,83,79,324
수현,3,여자,63,60,31,71,225
호영,4,남자,75,50,90,88,303
영호,4,남자,80,88,91,72,331
용식,2,남자,82,88,79,90,339
나영,1,여자,90,92,81,95,358
석영,1,남자,91,90,89,80,350


- `데이터프레임.loc[]`로 데이터 추출 + 기술 통계값 관련 함수 사용

In [ ]:
print(df)
# 평균
df.loc[:, '국어':].mean(axis=1)

# 최대
df.loc[:, '국어':].max(axis=1)

# 최소
df.loc[:, '국어':].min(axis=1)

# 표준편차
df.loc[:, '국어':].std(axis=1)

# 분산
df.loc[:, '국어':].var(axis=1)


    학년  성별  국어   영어  수학  과학
이름                         
철수   1  남자  98   77  88  64
영희   2  여자  88  120  62  72
민철   1  남자  92   70  83  79
수현   3  여자  63   60  31  71
호영   4  남자  75   50  90  88
영호   4  남자  80   88  91  72
용식   2  남자  82   88  79  90
나영   1  여자  90   92  81  95
석영   1  남자  91   90  89  80


이름
철수    213.583333
영희    643.666667
민철     83.333333
수현    304.916667
호영    338.916667
영호     72.916667
용식     26.250000
나영     36.333333
석영     25.666667
dtype: float64

#### 3) 조건별 파생변수

- `numpy.where(조건부, True시출력, False시출력)`
- `numpy.select(조건부, True시출력, False시출력)`
- if ~ else


In [ ]:
# 합계와 평균에 대한 파생변수 추가
df5 = df.copy()
df5['합계'] = df.loc[:, '국어': '과학'].sum(axis = 1)
df5['평균'] = df.loc[:, '국어': '과학'].mean(axis = 1)
df5

# 평균값에 따른 조건부 파생변수
df5['결과'] = np.where(df5['평균'] >= 80, '합격', '불합격')
df5

# 경우의 수에 따른 조건값
# - 학점을 부여하기 위한 점수의 구간을 설정하는 조건들을 리스트로 설정
conditions = [(df5['평균'] >= 90),
              (df5['평균'] >= 80),
              (df5['평균'] >= 70)
              ]
# - 조건에 따라 부여될 학점
value = ['A', 'B', 'C']

df5['학점'] = np.select(conditions, value, default = 'F')
df5

,학년,성별,국어,영어,수학,과학,합계,평균,결과,학점
이름,,,,,,,,,,
철수,1,남자,98,77,88,64,327,81.75,합격,B
영희,2,여자,88,120,62,72,342,85.50,합격,B
민철,1,남자,92,70,83,79,324,81.00,합격,B
수현,3,여자,63,60,31,71,225,56.25,불합격,F
호영,4,남자,75,50,90,88,303,75.75,불합격,C
영호,4,남자,80,88,91,72,331,82.75,합격,B
용식,2,남자,82,88,79,90,339,84.75,합격,B
나영,1,여자,90,92,81,95,358,89.50,합격,B
석영,1,남자,91,90,89,80,350,87.50,합격,B


참조 :
- https://blog.naver.com/rising_n_falling/221631637822(열 삭제)